# Лабораторная работа №3 (Нейронные сети)

## Введение

В ходе данной лабораторной работе необходимо использовать библиотеку Keras. Данная библиотека предоставляет удобный API для создания и обучения нейронных сетей. Документацию по данной библиотеке можно найти по ссылке https://keras.io/

Все наборы данных находятся в модуле `keras.datasets`. Информацию об используемых в лабораторной работе наборах данных можно найти по ссылке https://keras.io/datasets/

## Задание

Доказано, что двухслойная нейронная сеть может успешно представить неразрывный выпуклый n-мерный многоугольник, а трехслойная нейронная сеть теоритечески способна аппроксимировать любые поверхности.  
К сожалению, универсальность нейронных сетей зачастую нивелируется сложностью правильной настройки её гиперпараметров и структуры. Лабораторная работа состоит в том, чтобы увидеть, как влияет выбор тех или иных гиперпараметров на поведение всей сети, и как понять, что гиперпараметр установлен неверно.

Задание состоит из следующих этапов:
1. Изучить базовый принцип работы нейронных сетей
2. Загрузить набор данных согласно варианту
3. Закодировать данные понятным для сети образом. Для задач классификации выходные признаки закодировать one-hot кодированием.
4. Используя интерактивный интерфейс, показать на графиках проблему переобучения нейронной сети*. Для этого нужно установить большое число слоев и нейронов в каждом слое. Переобучение возникает тогда, когда точности на тренировочной и тестовой выборках сильно разнятся между собой.
5. Используя интерактивный интерфейс, показать на графиках что происходит при неправильном выборе темпа обучения
6. Проделать то же самое для коэффициентов регуляризации.
7. Поигравшись с гиперпараметрами, найти, на Ваш взгляд, оптимальные параметры нейронной сети и зафиксировать точность итоговой модели. Точность оптимальной модели желательно должна быть не меньше 75%.

* -- выполнять данный пункт только при наличии GPU, совместимого с Tensorflow, т.к. обучение сети до состояния переобучения может занять слишком большое количество времени. Если видеокарта несовместима с tensorflow, можно попробовать backend Theano.

По умолчанию, tensorflow использует только CPU. Чтобы tensorflow использовал GPU, нужно (предварительно активировав окружение `activate ai`): 
1. Удалить обычный пакет tensorflow: `pip uninstall tensorflow`
2. Установить пакет tensorflow-gpu, NVIDIA CUDA и NVIDIA cuDNN, в соответствии с [инструкцией](https://www.tensorflow.org/install/gpu)

## Варианты

|Вариант|Набор данных|Команда|Способ кодирования|
|-|-|-|-|
|1|CIFAR10|`from keras.datasets import cifar10`|One-hot для выходных признаков. Входные представить как одномерный массив.|
|2|CIFAR100|`from keras.datasets import cifar100`|One-hot для выходных признаков. Входные представить как одномерный массив.|
|3|MNIST Digits|`from keras.datasets import mnist`|One-hot для выходных признаков. Входные представить как одномерный массив.|
|4|MNIST Fashion|`from keras.datasets import fashion_mnist`|One-hot для выходных признаков. Входные представить как одномерный массив.|
|5|Boston Housing|`from keras.datasets import boston_housing`|Кодирование не требуется (задача регрессии)|
|6*|IMDB|`from keras.datasets import imdb`|Выходные признаки оставить как есть. Входные -- Bag of Words (см. предыдущую л. р.)|
|7*|IMDB|`from keras.datasets import imdb`|Выходные признаки оставить как есть. Входные -- TF-IDF (см. предыдущую л. р.)

Варианты 6 и 7 объективно сложнее остальных и выбираются на усмотрение самими студентами.

## Содержание отчета

1. Титульный лист
2. Текст задания согласно варианту
3. Исходный код
4. Графики и пояснения к ним
5. График обучения оптимальной модели
6. Вывод по работе

## Вопросы к защите

1. Устройство одиночного нейрона (персептрон)
2. Сеть нейронов
3. Метод обратного распространения ошибки
4. Переобучение, паралич сети.
5. Функции активации
6. Виды регуляризации (L1, L2, ElasticNet)

## Инструкция

### Загрузка данных

In [1]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('Input shape:', x_train.shape)
print('Output shape:', y_train.shape)

ModuleNotFoundError: No module named 'keras'

### Кодирование признаков

In [8]:
import numpy as np

def one_hot(y):
    t = np.zeros((len(y), np.max(y) + 1))
    t[np.arange(len(y)), y] = 1
    return t

y_train, y_test = one_hot(y_train), one_hot(y_test)
x_train, x_test = x_train.reshape((x_train.shape[0], -1)), x_test.reshape((x_test.shape[0], -1))

print('Encoded input shape:', x_train.shape)
print('Encoded output shape:', y_train.shape)

Encoded input shape: (60000, 784)
Encoded output shape: (60000, 10)


### Интерактивный функционал

Для достижения оптимальной точности модели следует пользоваться следующими правилами:
* Если решается задача регрессии, то последний слой должен иметь линейную активационную функцию
* Если решается задача классификации, то последний слой может иметь активационную функцию softmax, sigmoid, tanh. softmax предпочтительна.
* Скрытые слои обычно имеют следующие активационные функции: relu, sigmoid, tanh. Обычно берется relu, затем, если точность не устраивает, можно попробовать две другие.
* Функция потерь выбирается следующим образом:
    * Если решается задача классификации:
        * Если классов ровно два, можно применять бинарную энтропию
        * Если больше, то применяется категориальная энтропия
        * Другие функции применять можно, но эффективность обучения резко снизится
    * Если решается задача регрессии, то берется С.К.О. или среднее абсолютное отклонение.
    
    
Для построения модели и графиков можно использовать скрипт, написанный ниже. Однако, при надобности, не запрещается писать собственный скрипт.

In [9]:
from matplotlib import pyplot as plt
%matplotlib inline
from ipywidgets import *

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import Callback
from keras import regularizers
from keras import optimizers

def plot_accuracy(history):
    # Plot training & validation accuracy values
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()


model = None
history = {'acc': [], 'val_acc': [], 'loss': [], 'val_loss': []}
style = {'description_width': 'initial'}
@interact_manual(
    n_layers=IntSlider(min=0, max=3, value=1, description='Скрытых слоев: ', style=style),
    layer1_size=IntSlider(min=0, max=20, value=6, description='Нейронов в слое 1: ', style=style),
    layer2_size=IntSlider(min=0, max=20, value=3, description='Нейронов в слое 2: ', style=style),
    layer3_size=IntSlider(min=0, max=20, value=0, description='Нейронов в слое 3: ', style=style),
    inner_layers_type=Dropdown(options=['softmax', 'relu', 'tanh', 'sigmoid', 'linear'], value='relu',
                          description='Активация скрытых слоев: ', style=style),
    layer_out_type=Dropdown(options=['softmax', 'relu', 'tanh', 'sigmoid', 'linear'], value='sigmoid',
                          description='Активация выходного слоя: ', style=style),
    loss_func=Dropdown(options={
        'Среднеквадратическое отклонение': 'mse', 
        'Среднее абсолютное отклонение': 'mae',
        'Бинарная энтропия': 'binary_crossentropy',
        'Категориальная энтропия': 'categorical_crossentropy'
    }, value='categorical_crossentropy', description='Функция потерь: ', style=style),
    lr=ToggleButtons(options=["-0.1", "0", "0.001", "0.01", "0.05", "0.1", "0.5", "1", "5"], 
                               value="0.01", description='Темп обучения: ', style=style),
    l1=ToggleButtons(options=["-0.1", "0", "0.0001", "0.0005", "0.001", "0.005", "0.01", "0.05", "0.1"], 
                               value="0.0001", description='Регуляризация L1: ', style=style),
    l2=ToggleButtons(options=["-0.1", "0", "0.0001", "0.0005", "0.001", "0.005", "0.01", "0.05", "0.1"], 
                               value="0.0001", description='Регуляризация L2: ', style=style),
    epochs=BoundedIntText(value=20, min=1, max=1000, step=1, description='Количество эпох: ', style=style),
    create_new_model=Checkbox(value=True, description='Создать новую модель: ', style=style)
)
def interactive_learning(n_layers, loss_func, lr, l1, l2, 
                         layer_out_type,
                         layer1_size, layer2_size, layer3_size, inner_layers_type,
                         create_new_model, epochs):
    layer_sizes = [layer1_size, layer2_size, layer3_size]
    lr, l1, l2 = float(lr), float(l1), float(l2)
    global model, history
    
    if create_new_model:
        model = Sequential()
        history = {'acc': [], 'val_acc': [], 'loss': [], 'val_loss': []}

        if n_layers == 0:
            model.add(Dense(len(y_train[0]), activation=layer_out_type, 
                            input_shape=x_train[0].shape, kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
        else:
            model.add(Dense(layer_sizes[0], activation=inner_layers_type, 
                            input_shape=x_train[0].shape, kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
            for i in range(1, n_layers):
                model.add(Dense(layer_sizes[i], activation=inner_layers_type,
                               kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
            model.add(Dense(len(y_train[0]), activation=layer_out_type, kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))

        model.compile(
         optimizer = optimizers.sgd(lr=lr),
         loss = loss_func,
         metrics = ["accuracy"]
        )
    
    for key, entry in model.fit(
     x_train, y_train,
     epochs=epochs,
     validation_data=(x_test, y_test),
     verbose=1
    ).history.items():
        history[key].extend(entry)
    
    print('History: ', history)
    print('Accuracy: ', history['val_acc'][-1])
    plot_accuracy(history)


interactive(children=(IntSlider(value=1, description='Скрытых слоев: ', max=3, style=SliderStyle(description_w…